# AWS Lambda を MCP ツール化する
## Bedrock AgentCore Gateway を使用して AWS Lambda 関数をセキュアな MCP ツールに変換

## 概要
Bedrock AgentCore Gateway は、お客様が既存の AWS Lambda 関数を、インフラやホスティングを管理することなく完全管理された MCP サーバーに変換する方法を提供します。Gateway は、これらすべてのツールに対して統一された Model Context Protocol（MCP）インターface を提供します。Gateway は、受信リクエストとターゲットリソースへのアウトバウンド接続の両方に対してセキュアなアクセス制御を確保するため、デュアル認証モデルを採用しています。フレームワークは2つの主要コンポーネントで構成されています：ゲートウェイターゲットへのアクセスを試行するユーザーを検証・認可する Inbound Auth と、認証されたユーザーに代わってゲートウェイがバックエンドリソースに安全に接続できるようにする Outbound Auth です。Gateway は、アウトバウンド認証のために AWS Lambda 関数への呼び出しを認可するために IAM ロールを使用します。

![How does it work](images/lambda-iam-gateway.png)

### チュートリアル詳細


| 項目                | 詳細                                                      |
|:-------------------|:---------------------------------------------------------|
| チュートリアル タイプ | インタラクティブ                                            |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway Target タイプ | AWS Lambda                                                |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | AWS IAM                                                   |
| LLM モデル           | Anthropic Claude Sonnet 3.7、Amazon Nova Pro              |
| チュートリアル コンポーネント | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル領域   | 横断的                                                    |
| サンプルの複雑さ     | 簡単                                                      |
| 使用 SDK            | boto3                                                     |

チュートリアルの最初の部分では、いくつかの AmazonCore Gateway ターゲットを作成します

### チュートリアル アーキテクチャ
このチュートリアルでは、AWS Lambda 関数で定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーション目的として、Amazon Bedrock モデルを使用した Strands Agent を使用します。
この例では、get_order と update_order の2つのツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

## AgentCore Gateway 受信リクエスト用の認証設定
AgentCore Gateway は、インバウンドとアウトバウンドの認証を通じて安全な接続を提供します。インバウンド認証では、AgentCore Gateway は呼び出し時に渡された OAuth トークンを分析して、ゲートウェイ内のツールへのアクセスを許可または拒否することを決定します。ツールが外部リソースへのアクセスを必要とする場合、AgentCore Gateway は API キー、IAM、または OAuth トークンを介したアウトバウンド認証を使用して、外部リソースへのアクセスを許可または拒否できます。

インバウンド認証フロー中、エージェントまたは MCP クライアントが OAuth アクセストークン（ユーザーの IdP から生成）を追加して AgentCore Gateway の MCP ツールを呼び出します。AgentCore Gateway は OAuth アクセストークンを検証し、インバウンド認証を実行します。

AgentCore Gateway で実行されているツールが外部リソースにアクセスする必要がある場合、OAuth は Gateway ターゲット用のリソース認証情報プロバイダーを使用してダウンストリームリソースの認証情報を取得します。AgentCore Gateway は、呼び出し元にダウンストリーム API へのアクセス権を与えるために、認証認証情報を渡します。

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
os.environ['AWS_ACCESS_KEY_ID'] = '' # set the access key
os.environ['AWS_SECRET_ACCESS_KEY'] = '' # set the secret 
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1') # set the AWS region

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認証用 Amazon Cognito プールの作成

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# インバウンド認証用 Amazon Cognito オーソライザーを使用した Gateway の作成

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# AWS Lambda ターゲットを作成して MCP ツールに変換

In [ ]:
# Replace the AWS Lambda function ARN below
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Strands Agent からの Bedrock AgentCore Gateway の呼び出し

Strands エージェントは、Model Context Protocol（MCP）仕様を実装する Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合します。この統合により、AI エージェントと AWS サービス間のセキュアで標準化された通信が可能になります。

その核心となるのは、Bedrock AgentCore Gateway が基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠の Gateway として機能することです。これらの API により、任意の MCP 準拠クライアントまたは SDK が、セキュアで標準化された方法で利用可能なツールを発見し、操作することができます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は、[MCP Authorization 仕様](https://modelcontextprotocol.org/specification/draft/basic/authorization)に厳密に準拠し、堅牢なセキュリティとアクセス制御を確保します。これは、Strands エージェントによるすべてのツール呼び出しが認証ステップを通過し、強力な機能を有効にしながらセキュリティを維持することを意味します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。Gateway は、必要なすべてのセキュリティ検証、プロトコル変換、およびサービス相互作用を処理し、プロセス全体をシームレスでセキュアにします。

このアーキテクチャアプローチは、MCP 仕様を実装する任意のクライアントまたは SDK が Gateway を通じて AWS サービスと相互作用できることを意味し、AI エージェント統合のための多用途で将来性のあるソリューションとなっています。

![Strands agent calling Gateway](images/strands-lambda-gateway.png)

# インバウンド認証用 Amazon Cognito からのアクセストークンの要求

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Bedrock AgentCore Gateway を使用した AWS Lambda の MCP ツールを呼び出す Strands エージェント

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    # print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    agent("Check the order status for order id 123 and show me the exact response from the tool")
    # Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # You can replace this with unique identifier. 
    name=targetname+"___get_order_tool", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    arguments={"orderId": "123"}
    )
    # Print the MCP Tool response
    print(f"Tool Call result: {result['content'][0]['text']}")


**問題：下のセルを実行中に以下のエラーが発生する場合は、pydantic と pydantic-core のバージョンの非互換性を示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**解決方法：**

pydantic==2.7.2 と pydantic-core 2.27.2 の両方が互換性があることを確認する必要があります。完了後、カーネルを再起動してください。

# クリーンアップ

IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要があります。これは実行する例によって異なります。

## Gateway の削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)